<h2>Importing Data and Librarires</h2>

In [1]:
import numpy as np
import pandas as pd
from future.utils import iteritems
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import sys

if not sys.warnoptions: # ignore warnings 3ahsan e7na rewsheen gedan
    import warnings
    warnings.simplefilter("ignore")

In [2]:
gtd17 = pd.read_csv('globalterrorismdb_0718dist.csv',encoding='iso-8859-1',low_memory=False)
W = pd.read_csv('kaggle.csv',encoding='iso-8859-1',low_memory=False)


In [11]:
df = gtd17
df2 = W
names = {}
for name in df2.columns:
    names[name] = str(name)+'_w'

df2 = df2.rename(columns=names)
df.country_txt.astype(str)
df2.Id_w.astype(str)
join = pd.merge(df,df2, left_on=['country_txt'] ,right_on=['Id_w'] ,how='outer')

<h2>Preprocessing GTD + WHR</h2>

In [12]:
# MENA regions only
ds = join[join['region_txt'] == 'Middle East & North Africa']
# Year 2000-2017
ds = ds[ds['iyear'] > 1999]
# removing incidents where group is unknown
ds = ds[ds['gname'] != 'Unknown']
# choosing top groups
votes = {}
for name in ds['gname']:
    votes[name] = votes.get(name,0)+1

counts = []
names = []
for name,count in iteritems(votes):
    names.append(name)
    counts.append(count)
    
names = [name for _,name in sorted(zip(counts,names))]
names = names[-5:]
contained = [name in names for name in ds['gname']]
ds = ds[contained]
# encoding gname
lb_make = LabelEncoder()
ds["gname"] = lb_make.fit_transform(ds["gname"])
# drop redundant col , for ex region and region_txt , drop region
names = list(set(ds))
drop = ['region','region_txt','eventid','gname']
for col in names:
    f = str(col)+'_txt'
    if (f) in names and f not in drop :
            drop.append(f) # for ex if 


<h2>Split train: incidents 2000-2016 , test: incidents 2017</h2>

In [13]:
Xtrain = ds[ds['iyear']<2017]
Ytrain = Xtrain['gname']
Xtrain = Xtrain.drop(columns=drop)
Xtest = ds[ds['iyear']==2017]
Ytest = Xtest['gname']
Xtest = Xtest.drop(columns=drop)


<h2>Getting best features</h2>

In [14]:
votes = {}
for name in Ytrain:
    votes[name] = votes.get(name,0)+1

print('number of attakcs per each group in train ' ,votes)
votes = {}
for name in Ytest:
    votes[name] = votes.get(name,0)+1

print('number of attakcs per each group in test ' ,votes)

number of attakcs per each group in train  {0: 637, 3: 4271, 4: 1068, 1: 973, 2: 904}
number of attakcs per each group in test  {3: 1292, 4: 155, 2: 158, 1: 44}


In [15]:
def get_features(model , Xtrain , Ytrain):
    d = Xtrain.shape[1]
    num = len(Xtrain.columns) # total number of features
    best_scores = []
    names = list(Xtrain.columns) # names of all features
    Y = Ytrain 
    best_scores = []
    good_features = [] # stores the names of the best selected features
    bad_features = [] # stores the names of the  bad features not to try with again
    for i in range(5) : # take best 5 features
        scores = []
        
        for j,col in enumerate(Xtrain): # search for the next best feature that is the one which increases accuracy the most
            percent = round((i*d+j)/(d*5)*100,0)
            print('Loading '+str(percent)+' %',end="\r")
           
            if col in good_features or col in bad_features: # if feature reviewed before as good or bad skip
                scores.append(0)
                continue
            feature = Xtrain[col]  # get each feature
            X = Xtrain[good_features]  # get previous selected features
            #X.loc[:,col] = feature   # add this feature
            X = X.assign(col=feature)
            if 'O' in list(set(X.dtypes)): # one of the features is not numeric and needs encoding , note that nan values become zeros
                X = pd.get_dummies(X)
            X = X.fillna(X.mean()) # replace nan values by mean in numeric features
            if len (X.columns) > 20: # one-hot encoding results in too many dimensions , skip
                scores.append(0)
                continue
            if len (X.columns) == 0: # dont know why this is here
                scores.append(0)
                continue
            if X.isnull().values.any(): # a feauture still has nan values , this occurs if it initially has only nan values , it may had values in the removed columns
                scores.append(0)
                continue
           
            model.fit(X,Y)

            score = model.score(X,Y)
            if score < 0.02: # features that score less than 2% are bad and should not be reconsidered
                scores.append(0)
                bad_features.append(col)
            else:
                scores.append(score)
                
        index = scores.index(max(scores)) # get best feature
        best_scores.append(max(scores)) # store best score for 1 to n features
        good_features.append(names[index])
    return (good_features,best_scores)

In [16]:
def get_score(model,good_features,ds): # return the score after selecting best features
    scores = []
    fscores = []
    for i in range(5):
        features=good_features.copy()
        features = features[:i+1] 
        selected = features.copy()
        if 'iyear' not in features:
            selected.append('iyear')
        selected.append('gname')
        df = ds[selected]
        df = pd.get_dummies(df)
        df = df.fillna(df.mean())

        Xtrain = df[df['iyear']<2017]
        Ytrain = Xtrain['gname']
        Xtrain = Xtrain.drop(columns=['iyear','gname']) # always drop iyear , since test and trai wikk akways have different years
        Xtest = df[df['iyear']==2017]
        Ytest = Xtest['gname']
        Xtest = Xtest.drop(columns=['iyear','gname'])
        model.fit(Xtrain,Ytrain)
        scores.append(model.score(Xtest,Ytest))
        fscores.append(f1_score(Ytest,model.predict(Xtest),average='weighted'))
    return (max(scores),max(fscores))

<h2>KNN</h2>

In [17]:
model = KNeighborsClassifier(n_neighbors=2)
good_features,best_scores = get_features(model,Xtrain,Ytrain)
print('good features ',good_features)
print('train scores :',best_scores)
score,fscore = get_score(model,good_features,ds)
print('test score ',score)
print('test fscore ',fscore)

good features  ['latitude', 'iyear', 'INT_IDEO', 'Forest area percentage of total land area 2012_w', 'INT_MISC']
train scores : [0.8619635807971476, 0.9442251368903604, 0.9913408888322934, 0.9955431045460333, 0.99732586272762]
test score  0.9745300181928441
test fscore  0.9770600303990042


<h2>Decision Trees</h2>

In [18]:
model = DecisionTreeClassifier()
good_features,best_scores = get_features(model,Xtrain,Ytrain)
print('good features ',good_features)
print('train scores :',best_scores)
score,fscore = get_score(model,good_features,ds)
print('test score ',score)
print('test fscore ',fscore)

good features  ['latitude', 'INT_IDEO', 'natlty1', 'iyear', 'imonth']
train scores : [0.8976187444288807, 0.9893034509104801, 0.99656182350694, 0.99923596077932, 0.9998726601298866]
test score  0.9824135839902971
test fscore  0.9830058143903957


<h2>Naive Bayes</h2>

In [19]:
model = GaussianNB() 
good_features,best_scores = get_features(model,Xtrain,Ytrain)
print('good features ',good_features)
print('train scores :',best_scores)
score,fscore = get_score(model,good_features,ds)
print('test score ',score)
print('test fscore ',fscore)

good features  ['Intimate or nonintimate partner violence ever experienced 2001-2011_w', 'INT_IDEO', 'Change forest percentable 1900 to 2012_w', 'iyear', 'longitude']
train scores : [0.805806698077168, 0.9710938494842736, 0.9851012351967401, 0.9890487711702534, 0.9904495097415]
test score  0.9769557307459066
test fscore  0.9792635646796695


<h2>SVM</h2>

In [20]:
model = LinearSVC() 
good_features,best_scores = get_features(model,Xtrain,Ytrain)
print(good_features)
print(best_scores)
score,fscore = get_score(model,good_features,ds)
print('test score ',score)
print('test fscore ',fscore)

['Id_w', 'INT_IDEO', 'dbsource', 'INT_LOG', 'gsubname3']
[0.8063160575576213, 0.9853559149369667, 0.9903221698713868, 0.9918502483127467, 0.9918502483127467]
test score  0.979381443298969
test fscore  0.9815067535237294


<h2>Random Forest</h2>

In [21]:
model = RandomForestClassifier() 
good_features,best_scores = get_features(model,Xtrain,Ytrain)
print(good_features)
print(best_scores)
score,fscore = get_score(model,good_features,ds)
print('test score ',score)
print('test fscore ',fscore)

['latitude', 'INT_IDEO', 'natlty1', 'iyear', 'imonth']
[0.8930345091048008, 0.9887940914300267, 0.9960524640264867, 0.99885394116898, 0.99961798038966]
test score  0.9818071558520315
test fscore  0.9825148531586689


<h2>Quadratic Discriminant Analysis</h2>

In [22]:
model = QuadraticDiscriminantAnalysis() 
good_features,best_scores = get_features(model,Xtrain,Ytrain)
print(good_features)
print(best_scores)
score,fscore = get_score(model,good_features,ds)
print('test score ',score)
print('test fscore ',fscore)

['Intimate or nonintimate partner violence ever experienced 2001-2011_w', 'iyear', 'ransom', 'imonth', 'targsubtype1']
[0.805806698077168, 0.8693492932637209, 0.9258881955940405, 0.9317458296192538, 0.9354386858525404]
test score  0.9205579138872043
test fscore  0.9249294594688634
